In [1]:
%pip install Pillow

Note: you may need to restart the kernel to use updated packages.


In [2]:
import hashlib
import os
from PIL import Image

def file_hash(filepath):
    with open(filepath, 'rb') as f:
        return hashlib.md5(f.read()).hexdigest()

def remove_duplicates(dir_path):
    if not os.path.exists(dir_path):
        print("Directory does not exist.")
        return

    hashes = {}
    duplicates = []

    file_list = [os.path.join(dir_path, file) for file in os.listdir(dir_path) if file.lower().endswith(('.png', '.jpg', '.jpeg'))]
    for img_path in file_list:
        # Open the image and convert it to RGB
        try:
            img = Image.open(img_path).convert('RGB')
            hash = file_hash(img_path)

            if hash in hashes:
                duplicates.append(img_path)
            else:
                hashes[hash] = img_path
        except IOError:
            print(f'Error handling the file {img_path}, it may not be an image or is corrupted.')

    # Remove duplicates
    for dup in duplicates:
        os.remove(dup)
        print(f'Removed duplicate: {dup}')

    print(f"Removed {len(duplicates)} duplicates.")

directory_path = r'Transformed_Datasets\NOT_Auguste_Cropped_224'
remove_duplicates(directory_path)

Removed duplicate: Transformed_Datasets\NOT_Auguste_Cropped_224\extracted_face_0 (1137).jpg_0.png
Removed duplicate: Transformed_Datasets\NOT_Auguste_Cropped_224\extracted_face_0 (1143).jpg_0.png
Removed duplicate: Transformed_Datasets\NOT_Auguste_Cropped_224\extracted_face_0 (116).jpg_0.png
Removed duplicate: Transformed_Datasets\NOT_Auguste_Cropped_224\extracted_face_0 (1167).jpg_0.png
Removed duplicate: Transformed_Datasets\NOT_Auguste_Cropped_224\extracted_face_0 (1171).jpg_0.png
Removed duplicate: Transformed_Datasets\NOT_Auguste_Cropped_224\extracted_face_0 (118).jpg_0.png
Removed duplicate: Transformed_Datasets\NOT_Auguste_Cropped_224\extracted_face_0 (119).jpg_0.png
Removed duplicate: Transformed_Datasets\NOT_Auguste_Cropped_224\extracted_face_0 (120).jpg_0.png
Removed duplicate: Transformed_Datasets\NOT_Auguste_Cropped_224\extracted_face_0 (1247).jpg_0.png
Removed duplicate: Transformed_Datasets\NOT_Auguste_Cropped_224\extracted_face_0 (1259).jpg_0.png
Removed duplicate: Trans

In [3]:
%pip install tqdm

In [4]:
import os
import shutil
from PIL import Image
from tqdm import tqdm

def crop_image(image_path):
    try:
        with Image.open(image_path) as img:
            width, height = img.size
            # Crop 25% from the top-left corner
            return img.crop((0, 0, width // 2, height // 2))
    except IOError:
        print(f"Unable to open {image_path}. It may not be an image or is corrupted.")
        return None

def compare_images(img1, img2):
    # This function assumes both images are the same size and mode
    if img1.size != img2.size or img1.mode != img2.mode:
        return 0
    # Calculate similarity using pixel-wise comparison
    pairs = zip(img1.getdata(), img2.getdata())
    if len(img1.getbands()) == 1:  # For grayscale images
        dif = sum(abs(p1 - p2) for p1, p2 in pairs)
    else:
        dif = sum(abs(c1 - c2) for p1, p2 in pairs for c1, c2 in zip(p1, p2))
    ncomponents = img1.size[0] * img1.size[1] * 3
    similarity = (1 - dif / 255.0 / ncomponents) * 100
    return similarity

def find_similar_images(dir_path):
    if not os.path.exists(dir_path):
        print("Directory does not exist.")
        return

    similar_dir = os.path.join(dir_path, 'TOO_SIMILAR')
    if not os.path.exists(similar_dir):
        os.makedirs(similar_dir)

    files = [f for f in os.listdir(dir_path) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
    reference_image_path = os.path.join(dir_path, files[0])
    reference_image = crop_image(reference_image_path)

    if reference_image is None:
        print(f"Skipping file {files[0]} due to load error.")
        return

    for file in tqdm(files[1:], desc="Analyzing Similarity"):
        current_image_path = os.path.join(dir_path, file)
        current_image = crop_image(current_image_path)
        if current_image is not None:
            similarity = compare_images(reference_image, current_image)
            if similarity > 80:
                # Move similar image to 'TOO_SIMILAR'
                shutil.move(current_image_path, os.path.join(similar_dir, file))
                print(f"Moved {file} to TOO_SIMILAR - Similarity: {similarity:.2f}%")

directory_path = r'Transformed_Datasets\NOT_Auguste_Cropped_224'
find_similar_images(directory_path)


Analyzing Similarity:   0%|          | 11/11826 [00:00<04:08, 47.63it/s]

Moved extracted_face_0 (1).jpg_0.png to TOO_SIMILAR - Similarity: 81.24%
Moved extracted_face_0 (10).jpeg_0.png to TOO_SIMILAR - Similarity: 81.26%
Moved extracted_face_0 (100).jpg_0.png to TOO_SIMILAR - Similarity: 80.13%
Moved extracted_face_0 (1001).jpg_0.png to TOO_SIMILAR - Similarity: 80.08%


Analyzing Similarity:   0%|          | 21/11826 [00:00<04:49, 40.71it/s]

Moved extracted_face_0 (1005).jpg_0.png to TOO_SIMILAR - Similarity: 88.35%
Moved extracted_face_0 (1012).jpg_0.png to TOO_SIMILAR - Similarity: 83.80%


Analyzing Similarity:   0%|          | 31/11826 [00:00<04:49, 40.80it/s]

...

Moved extracted_face_Studio Photo Bordeaux (16).jpg_0.png to TOO_SIMILAR - Similarity: 81.60%
Moved extracted_face_Studio Photo Bordeaux (24).jpg_0.png to TOO_SIMILAR - Similarity: 81.75%
Moved extracted_face_Studio Photo Bordeaux (24).jpg_1.png to TOO_SIMILAR - Similarity: 84.00%
Moved extracted_face_Studio Photo Bordeaux (25).jpg_0.png to TOO_SIMILAR - Similarity: 82.97%
Moved extracted_face_Studio Photo Bordeaux (26).jpg_0.png to TOO_SIMILAR - Similarity: 85.50%


Analyzing Similarity: 100%|█████████▉| 11809/11826 [03:25<00:00, 73.85it/s]

Moved extracted_face_Studio Photo Bordeaux (33).jpg_0.png to TOO_SIMILAR - Similarity: 80.21%
Moved extracted_face_Studio Photo Bordeaux (34).jpg_0.png to TOO_SIMILAR - Similarity: 82.20%
Moved extracted_face_Studio Photo Bordeaux (35).jpg_0.png to TOO_SIMILAR - Similarity: 84.04%
Moved extracted_face_Studio Photo Bordeaux (37).jpg_0.png to TOO_SIMILAR - Similarity: 80.60%
Moved extracted_face_Studio Photo Bordeaux (38).jpg_0.png to TOO_SIMILAR - Similarity: 82.53%


Analyzing Similarity: 100%|██████████| 11826/11826 [03:26<00:00, 57.39it/s]

Moved extracted_face_Studio Photo Bordeaux (41).jpg_0.png to TOO_SIMILAR - Similarity: 84.29%
Moved extracted_face_Studio Photo Bordeaux (43).jpg_0.png to TOO_SIMILAR - Similarity: 81.28%
Moved extracted_face_Studio Photo Bordeaux (44).jpg_0.png to TOO_SIMILAR - Similarity: 82.27%
Moved extracted_face_The_greatest_picture.jpg_0.png to TOO_SIMILAR - Similarity: 84.02%
Moved extracted_face_The_greatest_picture.jpg_2.png to TOO_SIMILAR - Similarity: 83.43%
Moved extracted_face_The_greatest_picture.jpg_3.png to TOO_SIMILAR - Similarity: 89.49%
Moved extracted_face_The_greatest_picture.jpg_4.png to TOO_SIMILAR - Similarity: 88.45%
Moved extracted_face_The_greatest_picture.jpg_5.png to TOO_SIMILAR - Similarity: 86.37%
Moved extracted_face_The_greatest_picture.jpg_7.png to TOO_SIMILAR - Similarity: 83.25%
